In [4]:
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
import pickle
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import math

In [5]:
web3 = Web3(Web3.HTTPProvider("https://bsc-dataseed.binance.org/"))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)
web3.isConnected()

True

In [6]:
# Address = token contract
apikey = "GKDPH6RRX5YSH1TYPMNHNVKGMB6I9V3I8T"
address = "0xF700D4c708C2be1463E355F337603183D20E0808"

# Methods
approval_method = "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925"
transfer_method = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
stake_method = "0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c"


# Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

# Set topic0
topic0 = approval_method

# Current block
currentblock = web3.eth.blockNumber
endblock = currentblock

# Otros contratos
claim_contract = "0x739e6e290f9aa6ef3a929950aa47795b1d187229"
stake_contract = "0xdf5c4c1b98a4d045573ab2005d77e87daa58c853"


# Liquidity Pool
pcs = "0x00000000000000000000000072121d60b0e2f01c0fb7fe32ca24021b42165a40"
pcs_address = "0x72121d60b0e2F01c0FB7FE32cA24021b42165A40"

ape = "0x000000000000000000000000cC3A3Bc1d76Df321f94716E88224638C439267aa"
ape_address = "0xcC3A3Bc1d76Df321f94716E88224638C439267aa"

biswap = "0x000000000000000000000000B4c88543Bd95617Fb61F3d031B8E8E19aF33201E"
biswap_address = "0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E"

# Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

# Set topic0
topic0 = approval_method

# Wallets to check
check_wallets = [
    address,
    claim_contract,
    stake_contract,
    pcs_address,
    ape_address,
    biswap_address,
]

In [7]:
# Get ABI

url = "https://api.bscscan.com/api"
params = {
    "module": "contract",
    "action": "getabi",
    "address": address,
    "apikey": apikey,
}

response = requests.get(url, params=params)

if response.status_code == 200:
    abi = response.json()["result"]
    print("Abi recibida correctamente")
else:
    print("Error en la solicitud")

Abi recibida correctamente


In [8]:
w3address = Web3.toChecksumAddress(address.lower())
contract = web3.eth.contract(address=w3address, abi=abi)

token = contract.functions.name().call()
symbol = contract.functions.symbol().call()

print(f"Abi del token: {token}, con simbolo: {symbol}, recibida correctamente")

Abi del token: Galactic Quadrant, con simbolo: GQ, recibida correctamente


In [9]:
# Get "Internal Transactions" by Block Range
def get_internal_transactions(endblock, numberofblocks):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "txlistinternal",
        "startblock": endblock - numberofblocks,
        "endblock": endblock,
        "page": 1,
        "offset": 10,
        "sort": "asc",
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error en la solicitud"


# Get Logs by Address sin topic0
def get_logs_loop(address, endblock, blocks):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, endblock - loop * 1000)
        total_logs = total_logs + logs
    return total_logs


def get_logs1000(address, endblock):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"

# Get all logs
def get_all_logs_loop(address, endblock, fromblock):
    blocks = endblock-fromblock
    # if blocks % 1000 != 0:
    #     raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, fromblock + loop * 1000)
        if logs is not None:
            total_logs = total_logs + logs
    return total_logs


def get_all_logs1000(address, fromblock):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": fromblock,
        "toBlock": fromblock + 1000,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


# Get Logs by Address
def get_logstopic_loop(address, endblock, blocks, topic0):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, endblock - loop * 1000)
        total_logs = total_logs + logs
    return total_logs


def get_logs1000topic(address, endblock, topic0):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


# Get Logs by Address and From
def get_logstopic0_1_loop(address, endblock, blocks, topic0, topic1):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, endblock - loop * 1000)
        total_logs = total_logs + logs
    return total_logs


def get_logs1000topic0_1(address, endblock, topic0, topic1):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "topic0_1": "and",
        "topic1": topic1,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


# Get a list of 'BEP-20 Token Transfer Events' by wallet
def get_token_transfers(wallet, address):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": address,
        "address": wallet,
        "page": 1,
        "offset": 1000,
        "startblock": 0,
        "endblock": 999999999,
        "sort": "asc",
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error in the request"


# Remove known wallets
def delete_wallet_from_df(df, check_wallets):
    for wallet_address in check_wallets:
        result = df["wallets"].isin([wallet_address.lower()]).any()
        print(f"Esta {wallet_address} en las wallets? {result}")
        if result:
            df = df.query('wallets != "{}"'.format(wallet_address.lower()))
    return df

In [10]:
## Extractor de Events/topics0
total_logs = get_logs_loop(address, endblock, 100000)


# Save hash in a list
total_hash = []
for i in range(len(total_logs)):
    event_hash = total_logs[i]["topics"][0]
    total_hash.append(event_hash)

unique_event_hash = list(set(total_hash))


# Cantidad de logs extraidos
print(
    f"Se han obtenido {len(total_logs)} logs, en los cuales hay {len(unique_event_hash)} event hash unicos."
)
print(unique_event_hash)


# Guarda los nuevos event hash
logs_unique = []
for value in unique_event_hash:
    if value not in logs_unique:
        logs_unique.append(value)

print(
    f"Actualmente hay almacenados {len(logs_unique)} event hash unicos que utilizaremos para reconocer los diferentes methods de topic0"
)

Se han obtenido 12551 logs, en los cuales hay 3 event hash unicos.
['0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', '0x9d228d69b5fdb8d273a2336f8fb8612d039631024ea9bf09c424a9503aa078f0']
Actualmente hay almacenados 3 event hash unicos que utilizaremos para reconocer los diferentes methods de topic0


In [11]:
# Un dia son aprox 28200 bloques, queremos calcular el finde pasado -5/-7 dias
currentblock = web3.eth.blockNumber
endblock = currentblock 
fromblock = 15789191

# Get data
logs_weekend = get_all_logs_loop(address, endblock, fromblock)
token_tx = pd.DataFrame(logs_weekend)
token_tx.head()

KeyboardInterrupt: 

In [ ]:
token_tx.to_csv("gq_all_logs.csv", index=False)

In [ ]:
# TOKEN LOG CLEANING

# Data Clean
token_tx["timeStamp"] = token_tx["timeStamp"].apply(lambda x: int(x, 16))
token_tx["timeStamp"] = token_tx["timeStamp"].apply(lambda x: datetime.fromtimestamp(x))

df_topics = pd.DataFrame(token_tx["topics"].tolist(), columns=["method", "from", "to"])
token_tx = pd.concat([token_tx, df_topics], axis=1)

token_tx = token_tx.drop(
    columns=[
        "topics",
        "address",
        "data",
        "blockNumber",
        "blockHash",
        "gasPrice",
        "gasUsed",
        "logIndex",
        "gasUsed",
        "transactionIndex",
    ],
    axis=1,
)

token_tx.sort_values(by="timeStamp", ascending=True, inplace=True)

# Filtrar la fecha
token_tx["intime"] = 0
for index, row in token_tx.iterrows():
    if row["timeStamp"].dayofyear < 31 and row["timeStamp"].dayofyear > 28:
        token_tx.at[index, "intime"] = 1

token_tx.head()

token_tx["intime"] = token_tx["intime"].astype(bool)
token_tx = token_tx[token_tx["intime"] == 1]

token_tx = token_tx.drop(
    columns=["intime"],
    axis=1,
)

# Standarizar address, from & to
token_tx["to"] = "0x" + token_tx["to"].str[26:]
token_tx["from"] = "0x" + token_tx["from"].str[26:]

# Filtrar solo transferencias
token_tx = token_tx[token_tx["method"] == transfer_method]

In [ ]:
token_tx.head()

,timeStamp,transactionHash,method,from,to
811862,2023-01-29 00:04:18,0x6b7ca16a1dc279b6540ba156557714ce4e13ee986522...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xb88d09175676913d69924acac17ec69037f1551e,0x4571d47e9ed7d838e9af02ef7ae3f54c56650a40
811863,2023-01-29 00:05:18,0x0cc6b78ba8e98cfc7b562b215caf886673f75adc6df7...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xb4c88543bd95617fb61f3d031b8e8e19af33201e,0xf1ec5c64da312f96aef027c3dd26880b4e1a59b6
811864,2023-01-29 00:05:18,0xe7fc044ab8530a0c6ede611ad2b4fa499db181c57c69...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xb4c88543bd95617fb61f3d031b8e8e19af33201e
811865,2023-01-29 00:05:18,0x5cab2d07852db057169120f1f42ad0e5ef912ab0bd06...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xcc3a3bc1d76df321f94716e88224638c439267aa,0xef713415d4c058cb3cd5224174e28c9384d08e21
811866,2023-01-29 00:05:18,0x5cab2d07852db057169120f1f42ad0e5ef912ab0bd06...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xef713415d4c058cb3cd5224174e28c9384d08e21,0xb4c88543bd95617fb61f3d031b8e8e19af33201e


In [ ]:
wallets_1 = list(token_tx["from"].unique())
len(wallets_1)

992

In [ ]:
wallets_2 = list(token_tx["to"].unique())
len(wallets_2)

1393

In [ ]:
wallets = wallets_1 + wallets_2
wallets = pd.DataFrame(wallets, columns=["wallets"])
wallets = wallets.drop_duplicates()
wallets.shape

(1694, 1)

In [ ]:
wallets = delete_wallet_from_df(wallets, check_wallets)

Esta 0xF700D4c708C2be1463E355F337603183D20E0808 en las wallets? False
Esta 0x739e6e290f9aa6ef3a929950aa47795b1d187229 en las wallets? True
Esta 0xdf5c4c1b98a4d045573ab2005d77e87daa58c853 en las wallets? True
Esta 0x72121d60b0e2F01c0FB7FE32cA24021b42165A40 en las wallets? True
Esta 0xcC3A3Bc1d76Df321f94716E88224638C439267aa en las wallets? True
Esta 0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E en las wallets? True


In [ ]:
wallets = delete_wallet_from_df(wallets, check_wallets)

Esta 0xF700D4c708C2be1463E355F337603183D20E0808 en las wallets? False
Esta 0x739e6e290f9aa6ef3a929950aa47795b1d187229 en las wallets? False
Esta 0xdf5c4c1b98a4d045573ab2005d77e87daa58c853 en las wallets? False
Esta 0x72121d60b0e2F01c0FB7FE32cA24021b42165A40 en las wallets? False
Esta 0xcC3A3Bc1d76Df321f94716E88224638C439267aa en las wallets? False
Esta 0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E en las wallets? False


In [ ]:
wallets.to_csv("gq_all_wallets.csv", index=False)

In [40]:
wallets = pd.read_csv('gq_all_wallets.csv')

In [41]:
wallets_try = wallets[:3]
wallets_try.shape

(3, 1)

In [42]:


total_tx = []
for index, row in wallets_try.iterrows():
    wallet = row['wallets']
    wallets_tx = get_token_transfers(wallet, address)
    total_tx = total_tx + wallets_tx 



In [43]:
total_tx = pd.DataFrame(total_tx)
total_tx

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,23499109,1669801290,0x989ae51448d5b7cbaada8f644e8ba1788f906278ce35...,1380,0x0fa35d69077a64182afcaede941a694e8080b3c10e87...,0xb6af19068482ec5feb4c4ef7f81a4dfdfa0bbad2,0xf700d4c708c2be1463e355f337603183d20e0808,0xb88d09175676913d69924acac17ec69037f1551e,3335881575391497601387464,Galactic Quadrant,GQ,18,85,85050,5000000000,41700,7061991,deprecated,1899917
1,23500052,1669804123,0xd4b49a4fed7db80e08f33e80b88519fa73da0d52fb3a...,147,0x9e7699e6d84c3f2016b39b04d9590c55010a07785bc0...,0x170e7d670cd9260a70bcf00d915e05983d040305,0xf700d4c708c2be1463e355f337603183d20e0808,0xb88d09175676913d69924acac17ec69037f1551e,840567311251888613858,Galactic Quadrant,GQ,18,150,112035,5000000000,91343,24212646,deprecated,1898974
2,23500078,1669804201,0x0905c8c0151f9c34bd7e2f352f812eadff343151610b...,150,0xb8fc498e4b01e568374a90bb50a8a372db136eb38819...,0x170e7d670cd9260a70bcf00d915e05983d040305,0xf700d4c708c2be1463e355f337603183d20e0808,0xb88d09175676913d69924acac17ec69037f1551e,3994520037709685716394901,Galactic Quadrant,GQ,18,120,85682,5000000000,64990,11383261,deprecated,1898948
3,23500661,1669805976,0x6eaf5e484343457b07233ebb8ce192daacb4f36174b6...,151,0xabeb596399f71b4c67c1be867e964f8eb2e176452ef4...,0xb88d09175676913d69924acac17ec69037f1551e,0xf700d4c708c2be1463e355f337603183d20e0808,0x170e7d670cd9260a70bcf00d915e05983d040305,3995360605020937605008759,Galactic Quadrant,GQ,18,274,90503,5000000000,54813,24749185,deprecated,1898365
4,23501083,1669807242,0x7ed5efda378ef24b9fb19a018244b3b98ada10dd60ac...,18,0x874a1571f081caed06ce2d51f27a667b304aa2afaf46...,0x371848dd23d33bc615ba1e6d53045e5cecd48a43,0xf700d4c708c2be1463e355f337603183d20e0808,0xb88d09175676913d69924acac17ec69037f1551e,67116520258622566489516,Galactic Quadrant,GQ,18,37,168070,5000000000,91355,4152824,deprecated,1897943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,23919448,1671089240,0xadbb7ac19cfe22e89e23dbc9888724cc2e6dfeb01208...,96,0x166e4e437c109978e6a3a3693f8aa01c89cb7ee596ea...,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xf700d4c708c2be1463e355f337603183d20e0808,0x2ad6b5f8d411d127e43189fdffa01a256aaa1e98,680173793231145228968325,Galactic Quadrant,GQ,18,111,157753,5000000000,109801,12141333,deprecated,1479578
1068,24059615,1671515660,0x11b26c429541a81eb63f5d03c509ca82a2039b646547...,97,0x00b41a72acf8c639fb0b1ed43e40c06f85bef353c88f...,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xf700d4c708c2be1463e355f337603183d20e0808,0x2ad6b5f8d411d127e43189fdffa01a256aaa1e98,314140103438592776347311,Galactic Quadrant,GQ,18,71,157740,5000000000,121873,7930213,deprecated,1339411
1069,24062779,1671525196,0x899fee7b6dc3dcf3c9584350e0bac71f2a3df0ea65d5...,98,0xf5a422182dec5f419260a1e8f54c2c95a0a3c8e93ba0...,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xf700d4c708c2be1463e355f337603183d20e0808,0x2ad6b5f8d411d127e43189fdffa01a256aaa1e98,426199012640789124484417,Galactic Quadrant,GQ,18,80,157740,5000000000,121873,6951360,deprecated,1336247
1070,25189999,1674947148,0x44f24d8e989a02e04b738bfa01e08ad3de034878dadf...,99,0x0bda3d76ff5a52e8f65b319ab8ab201a0e62a2069bd6...,0x2ad6b5f8d411d127e43189fdffa01a256aaa1e98,0xf700d4c708c2be1463e355f337603183d20e0808,0x8b7a933211b6f2e9c64db48abfa84528e2399b31,100000000000000000000,Galactic Quadrant,GQ,18,9,56652,5010000000,56652,1056894,deprecated,209027


In [44]:
print(token_tx["timeStamp"].dtype)

int64


In [45]:
# Data Clean
token_tx = total_tx
token_tx["timeStamp"] = token_tx["timeStamp"].apply(lambda x: int(x, 16))
token_tx["timeStamp"] = token_tx["timeStamp"].apply(lambda x: datetime.fromtimestamp(x))

# df_topics = pd.DataFrame(token_tx["topics"].tolist(), columns=["method", "from", "to"])
# token_tx = pd.concat([token_tx, df_topics], axis=1)

token_tx = token_tx.drop(
    columns=[
        "blockNumber",
        "nonce",
        "blockHash",
        "contractAddress",
        "tokenName",
        "tokenSymbol",
        "tokenDecimal",
        "transactionIndex",
        "gas",
        "gasPrice",
        "gasUsed",
        "cumulativeGasUsed",
        "input",
        "confirmations",        
    ],
    axis=1,
)

token_tx.sort_values(by="timeStamp", ascending=True, inplace=True)


OSError: [Errno 22] Invalid argument

In [46]:
token_tx['timeStamp']

0       96259281552
1       96259293475
2       96259293697
3       96259299702
4       96259306050
           ...     
1067    96385667648
1068    96390436448
1069    96390500758
1070    96445165896
1071    96445203587
Name: timeStamp, Length: 1072, dtype: int64

In [51]:
token_tx['hash'][0]

'0x989ae51448d5b7cbaada8f644e8ba1788f906278ce35b1352cf42070160033da'

In [52]:
token_tx['timeStamp'][0]

96259281552

In [54]:

import datetime as dt
timestamp = 96259281552
readable_datetime = dt.datetime.fromtimestamp(timestamp)
print(readable_datetime)

OSError: [Errno 22] Invalid argument

In [56]:
timestamp = 96259281552
timestamp = timestamp / 10**12
readable_datetime = dt.datetime.fromtimestamp(timestamp)
print(readable_datetime)

1970-01-01 01:00:00.096259


In [57]:
timestamp = 96259281552 / 10**6
readable_datetime = dt.datetime.fromtimestamp(timestamp)
print(readable_datetime)

1970-01-02 03:44:19.281552
